# Importar Lib's

In [1]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact
from openpyxl import Workbook

# Passo 1: Listar arquivos PDF em uma pasta específica

In [9]:
diretorio_lote = r"C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\AC\1_0-CORRETIVA\1ªTRANCHE\2024"
#"C:\Users\HELY-DELL\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\1.0-CORRETIVA\2ªTRANCHE"

In [3]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\jupyter\OEM\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 19), step=1)

## Pasta contendo os arquivos PDF

In [10]:
def encontrar_duplicados(numocorrencia):
    duplicados = numocorrencia[numocorrencia.duplicated(subset='NUMOCORRENCIA')]
    return duplicados

arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

string = pd.DataFrame(pd.Series(arquivos_pdf).str.extract(r"(\d+)_(\d+)_(.+)_(.+)_(.+)_(.+)"))
string.rename(columns={1: 'NUMOCORRENCIA'}, inplace=True)
numocorrence = string[['NUMOCORRENCIA']].astype('float')  # Convertendo para float para acomodar NaN

# Contagem total de duplicatas
duplicadas_count = numocorrence[numocorrence.duplicated(subset='NUMOCORRENCIA')].shape[0]

# Encontrar duplicatas
date_duplicated= encontrar_duplicados(numocorrence.copy())  # Cria uma cópia independente
date_duplicated.dropna(inplace=True)  # Remove NaNs do DataFrame duplicatas
date_duplicated['NUMOCORRENCIA'] = date_duplicated['NUMOCORRENCIA'].astype('int')  # Convertendo de volta para int
# Mesclando os dados principais.
df_pdf = numocorrence.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
df_pdf = df_pdf[['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]
print("Número total de duplicatas:", duplicadas_count)
print("Duplicatas encontradas após remoção de NaN:")
display(df_pdf)

Número total de duplicatas: 0
Duplicatas encontradas após remoção de NaN:


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
0,2.024000e+11,645889.0,14622.0,892.0,ANTONIO GLEIDSON DA SILVA ARAUJO,1.0,VALIDADO,ONEOM002,2024-01-05 12:21:00
1,2.023000e+11,652076.0,14158.0,169303.0,ALUIZIO FERREIRA DE SOUZA,3.0,VALIDADO,ONEOM013,2024-01-05 15:48:00
2,2.024000e+11,652102.0,14157.0,1256.0,LUZIA BARRETO DE SOUZA,3.0,VALIDADO,ONEOM013,2024-01-06 15:31:00
3,2.024000e+11,652148.0,14330.0,2666.0,MARIA DO SOCORRO TEIXEIRA DA SILVA,8.0,VALIDADO,ONEOM001,2024-01-09 15:16:00
4,2.023000e+11,652390.0,14165.0,166982.0,ACLINES FERNANDES DA SILVA,3.0,VALIDADO,ONEOM013,2024-01-05 13:38:00
...,...,...,...,...,...,...,...,...,...
372,2.024000e+11,661020.0,13553.0,62616.0,BERNARDO JOSE DO NASCIMENTO FELIX,9.0,VALIDADO,ONEOM005,2024-04-27 17:21:00
373,2.024000e+11,661031.0,13648.0,47942.0,ESCOLA FRANCISCO TRAUMATURGO DE AZEVEDO,10.0,VALIDADO,ONEOM005,2024-04-29 10:49:00
374,2.024000e+11,661647.0,15395.0,59646.0,MARIA SANDRA OLIVEIRA DOS SANTOS,10.0,VALIDADO,ONEOM005,2024-04-29 06:48:00
375,2.024000e+11,669890.0,14015.0,40549.0,ANDERSSON NASCIMENTO DE SOUZA,3.0,VALIDADO,ONEOM013,2024-04-26 12:06:00


# Lista de arquivos PDF na pasta

# Passo 2: Ler o arquivo Excel com a lista de arquivos

In [11]:
caminho_arquivo_excel = df_filtered_global_oem
caminho_arquivo_excel['NUMOCORRENCIA'].count()
df_excel = caminho_arquivo_excel [['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

In [12]:
df_excel

,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
24,202400033793,661661.0,14427,74468,JOSE PEREIRA DA COSTA,8.0,VALIDADO,ONEOM001,2024-05-15 17:06:00
25,202400033785,646489.0,14373,74117,JOSE ALVES DE ABREU,8.0,VALIDADO,ONEOM001,2024-05-15 12:09:00
33,202400033439,642576.0,13725,74067,MARIA RITA COSTA DA SILVA,1.0,VALIDADO,ONEL004,2024-05-15 17:15:00
35,202400032233,679414.0,13856,73514,SORLIANE PINHEIRO DA SILVA,3.0,VALIDADO,ONEOM003,2024-05-11 17:33:00
36,202400032225,646414.0,13975,73289,MARIA DORISETE ALVES DE AZEVEDO BARROS,3.0,VALIDADO,ONEOM003,2024-05-12 10:31:00
...,...,...,...,...,...,...,...,...,...
2499,202300041326,652076.0,14158,169303,ALUIZIO FERREIRA DE SOUZA,3.0,VALIDADO,ONEOM013,2024-01-05 15:48:00
2503,202300041284,652186.0,14000,167025,EZILDO GOMES PINHEIRO,3.0,VALIDADO,ONEOM013,2024-01-13 09:26:00
2504,202300041276,652390.0,14165,166982,ACLINES FERNANDES DA SILVA,3.0,VALIDADO,ONEOM013,2024-01-05 13:38:00
2507,202300041243,656796.0,14007,165610,ANA MARIA XAVIER DE SOUZA,3.0,VALIDADO,ONEOM013,2024-01-07 08:39:00


## DataFrame a partir do arquivo Excel


#  Passo 3: Comparar os DataFrames para identificar arquivos ausentes no PDF e arquivos não listados no Excel

In [13]:
arquivos_ausentes_no_pdf = df_excel[~df_excel['NUMOCORRENCIA'].isin(df_pdf['NUMOCORRENCIA'])]
arquivos_nao_listados_no_excel = df_pdf[~df_pdf['NUMOCORRENCIA'].isin(df_excel['NUMOCORRENCIA'])]

## Resultados

In [14]:
print(arquivos_ausentes_no_pdf['NUMOCORRENCIA'].count())
display(arquivos_ausentes_no_pdf)

44


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
24,202400033793,661661.0,14427,74468,JOSE PEREIRA DA COSTA,8.0,VALIDADO,ONEOM001,2024-05-15 17:06:00
25,202400033785,646489.0,14373,74117,JOSE ALVES DE ABREU,8.0,VALIDADO,ONEOM001,2024-05-15 12:09:00
33,202400033439,642576.0,13725,74067,MARIA RITA COSTA DA SILVA,1.0,VALIDADO,ONEL004,2024-05-15 17:15:00
37,202400032167,661014.0,14559,70469,RAIMUNDO FARIAS DE OLIVEIRA,11.0,VALIDADO,ONEOM006,2024-05-11 15:28:00
38,202400032159,661687.0,14557,67588,DEOCLECIO RODRIGUES DOS SANTOS,11.0,VALIDADO,ONEOM006,2024-05-11 13:33:00
44,202400031813,657822.0,14923,69124,NAZIRA FIRMINO DA CUNHA,1.0,VALIDADO,ONEOM012,2024-05-08 15:31:00
45,202400031797,657990.0,13773,71972,MARLUCE CANUTO DA SILVA,2.0,VALIDADO,ONEOM014,2024-05-08 13:46:00
46,202400031763,652886.0,13439,70722,MARIA DA GLORIA DO NASCIMENTO SILVEIRA,1.0,VALIDADO,ONEOM012,2024-05-09 10:18:00
47,202400031755,642769.0,13750,70566,FRANCISCO COSTA TAVARES,1.0,VALIDADO,ONEL004,2024-05-15 17:39:00
48,202400031748,652779.0,14933,70510,DAMIÃO VIRICIO DA SILVA,1.0,VALIDADO,ONEOM012,2024-05-08 16:45:00


In [15]:
arquivos_nao_listados_no_excel['NUMOCORRENCIA'].count()
nao_listado_excel = arquivos_nao_listados_no_excel.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
nao_listado_excel

,NUMOS_x,UC_x,IDSIGFI_x,NUMOCORRENCIA,NOMECLIENTE_x,ROTA_x,STATUS_x,EXECUTOR_x,DTCONCLUSAO_x,Unnamed: 0,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG
0,NaN,NaN,NaN,3755.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,160976.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,160957.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,131956.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,154833.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,166329.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,32373.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,137812.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
recorrence= df_filtered_global_oem.merge(date_duplicated, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')
recorrence['IDSIGFI'] = recorrence['IDSIGFI'].astype('int')

In [17]:
duplicated_numocorrencia = date_duplicated.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')

In [18]:
print("listar arquivo duplicados:")
print(recorrence.shape)
display(recorrence)

print("listar nº ocorrencia que esta duplicada")
display(sorted(duplicated_numocorrencia['NUMOCORRENCIA']))  

print("Arquivos ausentes na lista de PDF:")
print(arquivos_ausentes_no_pdf.shape)
display(arquivos_ausentes_no_pdf)

print("\nArquivos não listados no Excel:")
print(arquivos_nao_listados_no_excel.shape)
display(arquivos_nao_listados_no_excel)

listar arquivo duplicados:
(427, 52)


,Unnamed: 0,ORIGEM,TIPO,NUMOS,NUMOCORRENCIA,ROTA,NOMECLIENTE,CPF,CODIGOENERGISA,UC,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG
0,False,ENERGISA,CORRETIVA,202400033793,74468,8.0,JOSE PEREIRA DA COSTA,709.851.542-68,MLPA7793128-22,661661.0,...,24,2024-05-15 16:34:00,2024-05-15 16:34:00,2024-05-10 00:00:00,3,2024-05-15,202400033793,Report2,2666,NaN
1,False,ENERGISA,CORRETIVA,202400033785,74117,8.0,JOSE ALVES DE ABREU,030.644.332-53,MLPA7749421-22,646489.0,...,34,2024-05-15 11:57:00,2024-05-15 11:57:00,2024-05-10 00:00:00,3,2024-05-15,202400033785,Report2,2666,NaN
2,False,ENERGISA,CORRETIVA,202400033439,74067,1.0,MARIA RITA COSTA DA SILVA,664.768.792-00,MLPA7188666-22,642576.0,...,34,2024-05-15 16:50:00,2024-05-15 16:50:00,2024-05-10 00:00:00,3,2024-05-15,202400033439,Report2,1500,NaN
3,False,ENERGISA,CORRETIVA,202400032233,73514,3.0,SORLIANE PINHEIRO DA SILVA,081.198.142-82,MLPA7504781-22,679414.0,...,34,2024-05-11 16:36:00,2024-05-11 16:36:00,2024-05-11 00:23:00,3,2024-05-11,202400032233,Report2,2666,NaN
4,False,ENERGISA,CORRETIVA,202400032225,73289,3.0,MARIA DORISETE ALVES DE AZEVEDO BARROS,702.833.462-72,MLPA7461274-22,646414.0,...,34,2024-05-12 09:59:00,2024-05-12 09:59:00,2024-05-11 00:23:00,3,2024-05-12,202400032225,Report2,2666,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,False,ENERGISA,CORRETIVA,202300041326,169303,3.0,ALUIZIO FERREIRA DE SOUZA,021.890.632-30,MLPA7582416-22,652076.0,...,24,2024-01-05 15:39:00,2024-01-05 15:39:00,2023-12-28 18:27:00,3,2024-01-05,202300041326,Report2,2666,NaN
423,False,ENERGISA,CORRETIVA,202300041284,167025,3.0,EZILDO GOMES PINHEIRO,666.463.402-34,MLPA7514808-22,652186.0,...,24,2024-01-13 08:16:00,2024-01-13 08:16:00,2023-12-28 18:20:00,3,2024-01-13,202300041284,Report2,2666,NaN
424,False,ENERGISA,CORRETIVA,202300041276,166982,3.0,ACLINES FERNANDES DA SILVA,70179726277,MLPA7582423-22,652390.0,...,24,2024-01-05 13:11:00,2024-01-05 13:11:00,2023-12-28 18:17:00,3,2024-01-05,202300041276,Report2,2666,NaN
425,False,ENERGISA,CORRETIVA,202300041243,165610,3.0,ANA MARIA XAVIER DE SOUZA,648.877.152-34,MLPA7514816-22,656796.0,...,24,2024-01-07 07:48:00,2024-01-07 07:48:00,2023-12-28 18:13:00,3,2024-01-07,202300041243,Report2,2666,NaN


listar nº ocorrencia que esta duplicada


[]

Arquivos ausentes na lista de PDF:
(58, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
24,202400033793,661661.0,14427,74468,JOSE PEREIRA DA COSTA,8.0,VALIDADO,ONEOM001,2024-05-15 17:06:00
25,202400033785,646489.0,14373,74117,JOSE ALVES DE ABREU,8.0,VALIDADO,ONEOM001,2024-05-15 12:09:00
33,202400033439,642576.0,13725,74067,MARIA RITA COSTA DA SILVA,1.0,VALIDADO,ONEL004,2024-05-15 17:15:00
37,202400032167,661014.0,14559,70469,RAIMUNDO FARIAS DE OLIVEIRA,11.0,VALIDADO,ONEOM006,2024-05-11 15:28:00
38,202400032159,661687.0,14557,67588,DEOCLECIO RODRIGUES DOS SANTOS,11.0,VALIDADO,ONEOM006,2024-05-11 13:33:00
44,202400031813,657822.0,14923,69124,NAZIRA FIRMINO DA CUNHA,1.0,VALIDADO,ONEOM012,2024-05-08 15:31:00
45,202400031797,657990.0,13773,71972,MARLUCE CANUTO DA SILVA,2.0,VALIDADO,ONEOM014,2024-05-08 13:46:00
46,202400031763,652886.0,13439,70722,MARIA DA GLORIA DO NASCIMENTO SILVEIRA,1.0,VALIDADO,ONEOM012,2024-05-09 10:18:00
47,202400031755,642769.0,13750,70566,FRANCISCO COSTA TAVARES,1.0,VALIDADO,ONEL004,2024-05-15 17:39:00
48,202400031748,652779.0,14933,70510,DAMIÃO VIRICIO DA SILVA,1.0,VALIDADO,ONEOM012,2024-05-08 16:45:00



Arquivos não listados no Excel:
(8, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
38,NaN,NaN,NaN,3755.0,NaN,NaN,NaN,NaN,NaT
148,NaN,NaN,NaN,160976.0,NaN,NaN,NaN,NaN,NaT
152,NaN,NaN,NaN,160957.0,NaN,NaN,NaN,NaN,NaT
161,NaN,NaN,NaN,131956.0,NaN,NaN,NaN,NaN,NaT
165,NaN,NaN,NaN,154833.0,NaN,NaN,NaN,NaN,NaT
179,NaN,NaN,NaN,166329.0,NaN,NaN,NaN,NaN,NaT
242,NaN,NaN,NaN,32373.0,NaN,NaN,NaN,NaN,NaT
308,NaN,NaN,NaN,137812.0,NaN,NaN,NaN,NaN,NaT


In [20]:
#Salvando a planilha Manutenções Energisa_preventiva para ser enviadas em Excell( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\OEM\output\csv_ausentes_corretivas_1_TRANCHE-geral.xlsx")
arquivos_ausentes_no_pdf.to_excel(dataframe_excel)
dataframe_excel.close()